# Instructions

This notebook is a prototype initial implementation of an MPC Tensor over IPFS's pubsub sockets. Run the Alice notebook first to start a server (which should be in the same folder as this one).

- installed IPFS (https://ipfs.io/docs/install/)
- run the command `ipfs daemon --enable-pubsub-experiment`
- run `python3 setup.py install` from the root directory of the OpenMined/Grid project (this project)

Then you're ready to run this notebook!

In [227]:
from grid import ipfsapi
import base64
import random
import torch
import keras
import json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np 
from grid.grid import Grid

In [441]:
BASE = 10
KAPPA = 9 # ~29 bits

PRECISION_INTEGRAL = 2
PRECISION_FRACTIONAL = 7
PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
BOUND = BASE**PRECISION

# Q field
Q = 6497992661811505123# < 64 bits
Q_MAXDEGREE = 2
assert Q > BASE**(PRECISION * Q_MAXDEGREE) # supported multiplication degree (without truncation)
assert Q > 2*BOUND * BASE**KAPPA # supported kappa when in positive range 

# P field
P = 1802216888453791673313287943102424579859887305661122324585863735744776691801009887 # < 270 bits
P_MAXDEGREE = 9
assert P > Q
assert P > BASE**(PRECISION * P_MAXDEGREE)

class MPCTensor(object):
    
    def __init__(self,grid,json_str=None,value=None,public=None,private=None,share=None,field=Q,id=None,channel=None):
        
        if(json_str is None):
            if(value is not None or private is not None):
                self.is_owner = True
            else:
                self.is_owner = False

            self._share = share
            self.field = field
            self.value = value
            self.grid = grid
            self.precision_fractional=PRECISION_FRACTIONAL

            if(id is None):
                id = str(random.randint(0,1000000))
                
            self.channel = channel

            self.id = str(id)
        else:
            self.deserialize(json_str)
            self.channel = channel
    
    def serialize(self):
        
        d = {}
        if(self.value is not None):
            d['v'] = self.value.tolist()
        if(self._share is not None):
            d['_share'] = self._share.tolist()
        
        d['id'] = self.id
        d['f'] = self.field
        d['p'] = self.precision_fractional
        d['o'] = self.is_owner
        
        return json.dumps(d)
    
    def __str__(self):
        return self.serialize()
    
    def deserialize(self,json_encoding):
        
        d = json.loads(json_encoding)
        keys = d.keys()
        
        if('v' in keys):
            self.value = np.array(d['v'],dtype='object')
        else:
            self.value = None
            
        if('_share' in keys):
            self._share = np.array(d['_share'],dtype='object')
        else:
            self._share = None
        
        self.id = d['id']
        self.field = d['f']
        self.precision_fractional = d['p']
        self.is_owner = d['o']
        
    def value2encoded_(self):
        upscaled = (self.value * BASE**self.precision_fractional).astype('object')
        field_elements = upscaled % self.field
        self.encoded_value = field_elements
        return self.encoded_value
    
    def encoded2value_(self):
        mask = (self.encoded_value <= self.field/2).astype('object')
        
        true_value = self.encoded_value
        false_value = self.encoded_value - self.field
        
        upscaled = (mask * true_value) + ((1 - mask) * false_value)
        rational = upscaled / BASE**self.precision_fractional
        return rational
    
    def encoded2shares_(self):
        
        public = (np.random.rand(*self.value.shape) * self.field).astype('object')
        private = ((self.encoded_value - public) % self.field).astype('object')
        
        self._share = private
        self.share_is_private = True
        
        return (public,private)
    
    def shares2encoded_(self,shares):
        self.encoded_value = (shares[0] + shares[1]) % Q
        return self.encoded_value
    
    def value2shares(self):
        self.value2encoded_()
        return self.encoded2shares_()
    
    def shares2value(self,shares):
        self.shares2encoded_(shares)
        self.value = self.encoded2value_()
        return self.value
    
    def __add__(self,y,publish=True,z_id=None):
        
        if(z_id is None):
            z_id = np.random.randint(0,10000000)
        
        if(publish):
            command = {}
            command['cmd'] = 'add_elem'
            command['x'] = self.id
            command['y'] = y.id
            command['z'] = z_id
            grid.api.pubsub_pub(topic=self.channel,payload=json.dumps(command))

        
        new_share = (self._share + y._share) % (self.field * DIV_DEPTH)
            
        return MPCTensor(grid,share=new_share,id=z_id,channel=self.channel)
    
    def __mul__(self,y,publish=True,z_id=None):
        
        y = int(y)
        
        if(y >= 1):

            if(z_id is None):
                z_id = np.random.randint(0,10000000)

            if(publish):
                command = {}
                command['cmd'] = 'mult_scalar'
                command['x'] = self.id
                command['y'] = y
                command['z'] = z_id
                grid.api.pubsub_pub(topic=self.channel,payload=json.dumps(command))

            new_share = ((self._share * y).astype('object') % self.field).astype('object')

            return MPCTensor(grid,share=new_share,id=z_id,channel=self.channel,field=self.field)
        else:
            print("Cannot divide yet")
    
    def __sub__(self,y):
        if(self.is_owner and not y.is_owner):
            new_share = (self.private - y.public) % self.field
        elif(y.is_owner and not self.is_owner):
            new_share = (self.public - y.private) % self.field
        elif(not self.is_owner and not y.is_owner):
            new_share = (self.public - y.public) % self.field
        else:
            new_share = (self.private - y.private) % self.field
            
        return MPCTensor(grid,private=new_share)
        
    def reconstruct(self):

        def send_request():
            command = {}
            command['cmd'] = "send_tensor"
            command['id'] = str(self.id)

            grid.api.pubsub_pub(topic=self.channel,payload=json.dumps(command))

        def receive_tensor(message):
            command = json.loads(message['data'])
            if(command['cmd'] == 'receive_tensor'):
                tensor = MPCTensor(grid,json_str=command['data'],channel=self.channel)
                if(int(tensor.id) == int(self.id)):
                    return tensor

        y = grid.listen_to_channel(channel=self.channel,handle_message=receive_tensor,init_function=send_request,ignore_from_self=False)
        
        
        self.shares2value([y._share,self._share])
        return self
  
    def share(self,alice):
        
        self.channel = alice.channel
        
        public,private = self.value2shares()
        public_tensor = MPCTensor(self.grid,share=public,id=self.id)
        self._share = private
        
        command = {}
        command['cmd'] = 'receive_tensor_share'
        command['data'] = str(public_tensor)

        grid.api.pubsub_pub(topic=alice.channel,payload=json.dumps(command))
        
        return self
        
        
class MPCGrid(object):
    
    def __init__(self,grid,channel):
        
        self._tensors = {}
        self.grid = grid
        self.channel = channel
        
    def process_message(self,msg):

        command = json.loads(msg['data'])

        if('cmd' in command.keys()):

            if(command['cmd'] == "receive_tensor_share"):
                tensor = MPCTensor(self.grid,json_str=command["data"],channel=self.channel)
                if(tensor.id not in self._tensors.keys()):
                    self._tensors[tensor.id] = tensor
                    print("Received Tensor:" + str(tensor.id))
                else:
                    print("Ignoring Tensor: " + str(tensor.id) + " because I seem to already have a tensor with the same name." )
                
            if(command['cmd'] == "send_tensor_share"):
                
                tensor_to_share = self._tensors[command['id']]
                tensor_to_share.share(self)

            elif(command['cmd'] == 'add_elem'):
                print("Adding " + str(command['x']) + " + " + str(command['y']) + "-> " + str(command['z']))

                z = self._tensors[command['x']].__add__(self._tensors[command['y']],False,z_id=command['z'])

                self._tensors[z.id] = z
                
            elif(command['cmd'] == 'mult_scalar'):
                print("Multiplying " + str(command['x']) + " * " + str(command['y']) + "-> " + str(command['z']))

                z = self._tensors[command['x']].__mul__(float(command['y']),False,z_id=command['z'])

                self._tensors[z.id] = z

            elif(command['cmd'] == "send_tensor"):

                tensor_to_send = str(self._tensors[command['id']])

                command = {}
                command['cmd'] = 'receive_tensor'
                command['data'] = tensor_to_send

                grid.api.pubsub_pub(topic=self.channel,payload=json.dumps(command))

                
            elif(command['cmd'] == "what_tensors_are_available"):
                
                command = {}
                command['cmd'] = "available_tensors"
                
                available_tensors = list()
                for k,v in self._tensors.items():
                    if(v.value is not None):
                        available_tensors.append([k,v.value.shape])
                    elif(v._share is not None):
                        available_tensors.append([k,v._share.shape])
                
                command['available_tensors'] = available_tensors
                
                grid.api.pubsub_pub(topic=self.channel,payload=json.dumps(command))
                
    def work(self):
        self.grid.listen_to_channel(channel=self.channel,handle_message=self.process_message,ignore_from_self=False)
        
    def available_tensors(self):
        
        def send_request():
            command = {}
            command['cmd'] = "what_tensors_are_available"

            grid.api.pubsub_pub(topic=self.channel,payload=json.dumps(command))

        def receive_tensor(message):
            command = json.loads(message['data'])
            if(command['cmd'] == 'available_tensors'):
                return command['available_tensors']
        available_tensors = grid.listen_to_channel(channel=self.channel,handle_message=receive_tensor,init_function=send_request,ignore_from_self=False)
        return available_tensors
    
    def get_tensor_share(self,id):
        
        def send_request():
            command = {}
            command['cmd'] = "send_tensor_share"
            command['id'] = str(id)

            grid.api.pubsub_pub(topic=self.channel,payload=json.dumps(command))

        def receive_tensor(message):
            command = json.loads(message['data'])
            if(command['cmd'] == 'receive_tensor_share'):
                tensor = MPCTensor(grid,json_str=command['data'],channel=self.channel)
                if(str(tensor.id) == str(id)):
                    return tensor

        return grid.listen_to_channel(channel=self.channel,handle_message=receive_tensor,init_function=send_request,ignore_from_self=False)

    
    def get_tensor(self,id):
        
        def send_request():
            command = {}
            command['cmd'] = "send_tensor"
            command['id'] = str(id)

            grid.api.pubsub_pub(topic=self.channel,payload=json.dumps(command))

        def receive_tensor(message):
            command = json.loads(message['data'])
            if(command['cmd'] == 'receive_tensor'):
                tensor = MPCTensor(grid,json_str=command['data'],channel=self.channel)
                if(str(tensor.id) == str(id)):
                    return tensor

        return grid.listen_to_channel(channel=self.channel,handle_message=receive_tensor,init_function=send_request,ignore_from_self=False)

    def tensors(self):
        return self.available_tensors()
    
    def __repr__(self):
        tens = self.tensors()
        if(len(tens) < 10):
            s = "MPC Grid with Tensors:\n"
            for t in tens:
                s += "\t" + str(t) +"\n"
            return s
        return "< MPCGrid tensors:" + str(len(tens)) + " >"
    
    def __getitem__(self,id):
        return self.get_tensor_share(id)

In [442]:
grid = Grid()
grid.id

'Qma5efEQtBUxDonmSDajv9dZ52axH9MNULKTgVpXWK8Ak2'

In [443]:
alice = MPCGrid(grid,channel='bob <-> alice')

In [444]:
x = MPCTensor(grid,value=np.random.rand(3,3)).share(alice)
x.value

array([[ 0.71562871,  0.38243788,  0.61364252],
       [ 0.71547048,  0.77317717,  0.39275482],
       [ 0.22156455,  0.15446863,  0.12883901]])

In [445]:
y = x * 3

In [446]:
z = y * 2

In [447]:
y.reconstruct()
y.value

array([[2.1471232, 1.1472896, 1.8408448],
       [2.146304, 2.3192576, 1.1783168],
       [0.6647808, 0.4634624, 0.3863552]], dtype=object)

In [448]:
z.reconstruct()

In [449]:
z.value

array([[4.2942464, 2.2946816, 3.6816896],
       [4.292608, 4.6385152, 2.3566336],
       [1.3295616, 0.9269248, 0.7727104]], dtype=object)

In [450]:
alice

MPC Grid with Tensors:
	['xor_input', [4, 2]]
	['xor_output', [4]]
	['235769', [3, 3]]
	['1194836', [3, 3]]
	['1732279', [3, 3]]

In [451]:
a = MPCTensor(grid,value=np.random.rand(4,2)).share(alice)

In [452]:
b = alice['xor_input'] + a

In [453]:
b.reconstruct()

In [454]:
b.value

array([[0.804864, 0.6019072],
       [0.4882432, 1.3928448],
       [1.9214336, 0.8503296],
       [1.8274304, 1.9902464]], dtype=object)

In [455]:
a.value

array([[ 0.80484031,  0.60175367],
       [ 0.48812311,  0.39291783],
       [ 0.92140337,  0.85031995],
       [ 0.82741562,  0.99007357]])

In [456]:
x = MPCTensor(grid,value=np.random.rand(3,3))
y = MPCTensor(grid,value=np.random.rand(3,3))

In [457]:
x.share(alice)
y.share(alice)

In [458]:
z = x + y

In [459]:
z.reconstruct().value

array([[1.2230656, 1.1794432, 1.148928],
       [0.7968768, 0.9162752, 0.7938048],
       [0.6592512, 1.50016, 0.6889472]], dtype=object)

In [460]:
x.value + y.value

array([[ 1.22309238,  1.17944214,  1.1489424 ],
       [ 0.79693497,  0.91642486,  0.79380012],
       [ 0.65930805,  1.50011615,  0.68887243]])